In [ ]:
%reset

In [ ]:
import sys

sys.path.insert(0, '/home/users/ids29/DGRB')

In [ ]:
import aegis
import numpy as np
import torch
import healpy as hp
import pickle as pk
from astropy import units
from astropy import constants as c
import matplotlib.pyplot as plt
from os import listdir
import os
from sbi.inference import SNLE, SNPE#, prepare_for_sbi, simulate_for_sbi
from sbi import utils as utils
from sbi import analysis as analysis
# from sbi.inference.base import infer
from getdist import plots, MCSamples
from joblib import Parallel, delayed, parallel_backend
from scipy.integrate import quad, simpson
import pickle
from scipy.stats import norm
import sources.DMsignal as DMsignal
import sources.FermiBackgrounds as FermiBackgrounds

%matplotlib inline

In [ ]:
grains=1000
num_simulations = 240
num_workers = 48

In [ ]:
parameter_range_aegis = [[], []]
abundance_luminosity_and_spectrum_list = []
source_class_list = []
parameter_names = []
energy_range = [1000, 100000] #MeV 
energy_range_gen = [energy_range[0]*0.5, energy_range[1]*1.5]
# max_radius = 8.5 + 20*2 #kpc
max_radius = 220 #kpc #This is the virial radius of the Milky Way
exposure = 2000*10*0.2 #cm^2 yr
flux_cut = 1e-9 #photons/cm^2/s
angular_cut = np.pi #10*u.deg.to('rad') #degrees
angular_cut_gen = np.pi #angular_cut*1.5
lat_cut = 0 #2*u.deg.to('rad') #degrees
lat_cut_gen = lat_cut*0.5

In [ ]:
my_cosmology = 'Planck18'
# z_range = [0, 14]
luminosity_range = 10.0**np.array([33, 53]) # Minimum value set by considering source at distnace of closest approach by line of sight at 40 degrees and receiving 0.1 photon at detector side. CHANGE THIS FOR FINAL PROBLEM.
                                            # Maximum value set by that value of luminosity that suffieciently suppreses the luminoisity function at the higher end of the luminosity range.
# AEGIS object for Dark matter signal
def build_aegis_object():
    engine = aegis.aegis(abundance_luminosity_and_spectrum_list, source_class_list, parameter_range_aegis, energy_range, luminosity_range, max_radius, exposure, angular_cut, lat_cut, flux_cut, energy_range_gen=energy_range_gen, cosmology = my_cosmology, verbose = False)
    engine.angular_cut_gen, engine.lat_cut_gen = angular_cut_gen, lat_cut_gen
    return engine

my_AEGIS = build_aegis_object()


In [ ]:
# 1) Point to your data directory and choose a channel
data_dir = "/home/users/ids29/DGRB/data/dm_spectra/"         # must contain AtProduction_gammas.dat
channel  = "Tau"                           # e.g. 'b' (bottom quarks) or 'Tau'


# 2) Build the DMsignal object
dm = DMsignal.DMsignal(directory=data_dir, channel=channel)

In [ ]:
def DM_abundance(r, L, params): # Outputs the abundance function in the form dN/dV/dL. Note the dV.
    """
    Abundance function for the DM signal.
    """
    # r is the distance from the source # kpc
    # L is the luminosity of the source # photons/second
    # return the abundance function in the form dN/dV/dL in units of 1 / (kpc^3 photon/second)
    # params conatins 'mass of DM particle in GeV'


    A_DM = params[0]
    mass_DM_GeV = 200 #params[1] # UPDATE TO THE CURRENT PARAMS INDEX

    
    beta = 1.9
    A = 1.2e4 # MSolar^{-1} kpc^{-3}
    r_s = 21 #kpc
    r_tilde = r / r_s # unitless
    cross_section = 3e-26 #cm^3 s^{−1}
    
    def compute_integral_E(E_min: float, E_max: float, n_points: int = 10_001) -> float:
        """
        Compute ∫ dN/dE dE between E_min and E_max [MeV]
        by sampling on a uniform grid and applying Simpson's rule.
        n_points must be odd for Simpson's rule.
        """
        # make sure we have an odd number of points
        if n_points % 2 == 0:
            n_points += 1

        E = np.linspace(E_min, E_max, n_points)
        dNdE = dm.get_dNdE(
            desired_E_MeV=E,
            channel=channel,
            mass_MeV= mass_DM_GeV * 1e3
        )
        return simpson(dNdE, E)

    Emin, Emax = energy_range[0], energy_range[1]
    integral_E = compute_integral_E(Emin, Emax)



    Phi_pp = A_DM * cross_section / (8 * np.pi * mass_DM_GeV**2) * integral_E  # cm^3 s^{-1} GeV^{-2}

    
    def integrand_M_tilde(M_tilde):
        sigma = 0.74 + 0.011 * np.log(r/50) - 0.003 * M_tilde / 0.87
        Delta = np.log(L) - 77.4 + 0.23*np.log(r/50) - np.log((8*np.pi*Phi_pp)/(1e-28)) # unitless
        mu = sigma**2 * (1-beta) / 0.87  +  Delta
        return np.exp((mu**2 -Delta**2)/(2*sigma**2)) * np.exp(-(M_tilde - mu)**2/(2*sigma**2)) / (np.sqrt(2*np.pi)*sigma)
    


    def compute_integral_M_tilde_simpson(M_tilde_min: float, M_tilde_max: float, n_points: int = 10001) -> float:
        """
        Integrate integrand_M_tilde from M_min to M_max using Simpson's rule.
        n_points must be odd; if it's even, it's incremented by 1.
        """
        if n_points % 2 == 0:
            n_points += 1
        M_tilde = np.linspace(M_tilde_min, M_tilde_max, n_points)
        M_tilde_vec = M_tilde[:, None, None]  # add broadcast dims  # (10001, 1, 1)
        y = integrand_M_tilde(M_tilde_vec)
        print(f"y.shape: {y.shape}")
        return simpson(y, M_tilde, axis=0)


    # Example usage:
    M_min = 1 # MSolar
    M_tilde_min = 0.87 * np.log(M_min/1e5) # M_min = 1 MSolar
    integral_M_tilde = compute_integral_M_tilde_simpson(M_tilde_min, 14.0) # The integrand 'integrand_M_tilde' is sufficiently suppressed for all M_tilde > 14.0



    
    return 10**(5*(1-beta)) * A / (0.87 * L * r_tilde * (1+r_tilde)**2)   * integral_M_tilde # units of 1 / (kpc^3 photon/second)


def DM_abundance_dR(x, L, params): # Returns the abundance function in the form dN/dR/dL. Note the dR. # this function needs to be passed to AEGIS
    r = np.sqrt(x**2 + 8.5**2 - 2 * 8.5 * x * np.cos(np.deg2rad(40))) # x is the ad-hoc distance from the center of the Milky Way in kpc, and 8.5 is the distance of the Sun from the center of the Milky Way in kpc
    return 4*np.pi*r**2 * DM_abundance(r, L, params) # dV = 4*np.pi*r^2 dr, so we need to multiply by 4*np.pi*r^2


def DM_spectrum(energy, params): #'energy' is an array of energies in MeV 
    return dm.get_dNdE(
            desired_E_MeV=energy,
            channel=channel,
            mass_MeV= mass_DM_GeV * 1e3
        )

In [ ]:
# Add Fermi isotropic background source class
data_root = '/home/users/ids29/DGRB'
my_FB = FermiBackgrounds.FermiBackgrounds(data_root)
def spec_iso_wrap(energy, params):
    A_BG = params[0]
    iso_fit = my_FB.get_isotropic_background_spectrum_func()
    correction_normalization = 1.6168325498644325 # this factor adjusts the normalization such that the number of photons in this faux isotropic background is equal to the sum of number of photons in the Fermi isotropic background,
                               # and the number of photons in the Fermi non-isotropic background.
    return A_BG * iso_fit(energy) * correction_normalization


# input_params = torch.tensor([0])
# source_info_Fermi, photon_info_Fermi = simulator(input_params)
# print(f"Number of multi-photon sources = {source_info_Fermi['redshifts'].size}")
# print(f"Number of single-photon sources = {source_info_Fermi['single_p_redshifts'].size}")
# print(f"Number of photons = {photon_info_Fermi['energies'].size}")

In [ ]:
# Add Fermi isotropic background source class
data_root = '/home/users/ids29/DGRB'
my_FB = FermiBackgrounds.FermiBackgrounds(data_root)
def spec_iso_wrap(energy, params):
    A_BG = params[1]
    iso_fit = my_FB.get_isotropic_background_spectrum_func()
    correction_normalization = 1.6168325498644325 # this factor adjusts the normalization such that the number of photons in this faux isotropic background is equal to the sum of number of photons in the Fermi isotropic background,
                               # and the number of photons in the Fermi non-isotropic background.
    return A_BG * iso_fit(energy) * correction_normalization


# input_params = torch.tensor([0])
# source_info_Fermi, photon_info_Fermi = simulator(input_params)
# print(f"Number of multi-photon sources = {source_info_Fermi['redshifts'].size}")
# print(f"Number of single-photon sources = {source_info_Fermi['single_p_redshifts'].size}")
# print(f"Number of photons = {photon_info_Fermi['energies'].size}")

In [ ]:
# als_Poisson = [spec_poisson] # belongs to 'isotropic_diffuse' class
als_DM = [DM_abundance_dR, DM_spectrum]
als_FIB = [spec_iso_wrap]

# my_AEGIS_DM.abun_lum_spec = [als_DM]
# my_AEGIS_DM.source_class_list = ['isotropic_faint_single_spectrum']

# my_AEGIS_BG.abun_lum_spec =  [als_FIB] # Change when you want to include Poisson model.
# my_AEGIS_BG.source_class_list = ['isotropic_diffuse']

In [ ]:
# a simple simulator with the total number of photons as the summary statistic
def simulator(params):

    input_params = params.numpy()

    source_info = my_AEGIS.create_sources(input_params, grains=grains, epsilon=1e-2)
    photon_info = my_AEGIS.generate_photons_from_sources(input_params, source_info, grains=grains) 
    obs_info = {'psf_fits_path': '/home/users/ids29/DGRB/FERMI_files/psf_P8R3_ULTRACLEANVETO_V2_PSF.fits', 'edisp_fits_path': '/home/users/ids29/DGRB/FERMI_files/edisp_P8R3_ULTRACLEANVETO_V2_PSF.fits', 'event_type': 'PSF3', 'exposure_map': None}
    obs_photon_info = my_AEGIS.mock_observe(photon_info, obs_info)
    
    return obs_photon_info

In [ ]:
def manual_simulate_for_sbi(proposal, num_simulations=1000, num_workers=32):
    """
    Simulates the model in parallel using joblib.
    Each simulation call samples a parameter from the proposal and passes the index to the simulator.
    """
    def run_simulation(i):
        if i % 10 == 0:
            print(f"i= {i}")
        # Sample a parameter from the proposal (sbi.utils.BoxUniform has a .sample() method)
        theta_i = proposal.sample()
        photon_info = simulator(theta_i)

        with open(f'train_data_kerr_{i+920}.pkl', 'wb') as f:
            pickle.dump(photon_info, f)

        torch.save(theta_i, f'train_thetas_kerr_{i+920}.pt')


        # return theta_i , photon_info

    # Run simulations in parallel using joblib.
    # Switch to the threading backend
    with parallel_backend('threading', n_jobs=num_workers):
        Parallel(verbose=5, timeout=None)(delayed(run_simulation)(i) for i in range(num_simulations))

    # theta_list = zip(*results)

    # theta_tensor = torch.stack(theta_list, dim=0).to(torch.float32)
    
    
    # return theta_tensor #, photon_info_list

In [ ]:
# Define the prior using sbi.utils.BoxUniform
Phi_Poisson_training_range = [0, 1.0944]
Phi_SFG_training_range = [0, 0.1]

prior_range = torch.tensor([[Phi_Poisson_training_range[0], Phi_SFG_training_range[0]],
                            [Phi_Poisson_training_range[1], Phi_SFG_training_range[1]]])

prior = utils.BoxUniform(low=prior_range[0], high=prior_range[1])

manual_simulate_for_sbi(prior, num_simulations=num_simulations, num_workers=num_workers)

In [ ]:
# # 'photon_info_list' is a list of dictionaries

# # Save to file
# with open('train_data_Poisson_SFG_kerr_336.pkl', 'wb') as f:
#     pickle.dump(train_photon_info, f)

# # Save to file
# torch.save(train_thetas, 'train_thetas_Poisson_SFG_kerr_336.pt')
# torch.save(prior_range, 'prior_range_Poisson_SFG_kerr_336.pt')